# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 6 2023 8:30AM,256640,10,7547776,Emerginnova,Released
1,Feb 3 2023 2:49PM,256639,10,9107518-BO,Eywa Pharma Inc.,Released
2,Feb 3 2023 2:40PM,256638,10,PRF-51847,Bio-PRF,Released
3,Feb 3 2023 2:40PM,256638,10,PRF-51850,Bio-PRF,Released
4,Feb 3 2023 2:40PM,256638,10,PRF-51853,Bio-PRF,Released
5,Feb 3 2023 2:40PM,256638,10,PRF-51859,Bio-PRF,Released
6,Feb 3 2023 2:40PM,256638,10,PRF-51862,Bio-PRF,Released
7,Feb 3 2023 2:40PM,256638,10,PRF-51872,Bio-PRF,Released
8,Feb 3 2023 2:40PM,256638,10,PRF-51876,Bio-PRF,Released
9,Feb 3 2023 2:40PM,256638,10,PRF-51878,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,256634,Released,10
29,256635,Released,20
30,256638,Released,9
31,256639,Released,1
32,256640,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256634,NaN,NaN,10.0
256635,NaN,NaN,20.0
256638,NaN,NaN,9.0
256639,NaN,NaN,1.0
256640,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256580,0.0,0.0,1.0
256585,1.0,4.0,17.0
256592,3.0,1.0,5.0
256599,0.0,0.0,1.0
256600,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256580,0,0,1
256585,1,4,17
256592,3,1,5
256599,0,0,1
256600,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256580,0,0,1
1,256585,1,4,17
2,256592,3,1,5
3,256599,0,0,1
4,256600,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256580,,,1
1,256585,1,4,17
2,256592,3,1,5
3,256599,,,1
4,256600,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 6 2023 8:30AM,256640,10,Emerginnova
1,Feb 3 2023 2:49PM,256639,10,Eywa Pharma Inc.
2,Feb 3 2023 2:40PM,256638,10,Bio-PRF
11,Feb 3 2023 2:20PM,256635,10,ISDIN Corporation
31,Feb 3 2023 2:20PM,256634,10,ISDIN Corporation
41,Feb 3 2023 2:11PM,256633,10,"Digital Brands, LLC"
47,Feb 3 2023 2:00PM,256632,12,Livs Products
48,Feb 3 2023 1:59PM,256631,19,ACG North America LLC
50,Feb 3 2023 1:59PM,256630,10,ISDIN Corporation
62,Feb 3 2023 1:44PM,256628,19,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 6 2023 8:30AM,256640,10,Emerginnova,,,1
1,Feb 3 2023 2:49PM,256639,10,Eywa Pharma Inc.,,,1
2,Feb 3 2023 2:40PM,256638,10,Bio-PRF,,,9
3,Feb 3 2023 2:20PM,256635,10,ISDIN Corporation,,,20
4,Feb 3 2023 2:20PM,256634,10,ISDIN Corporation,,,10
5,Feb 3 2023 2:11PM,256633,10,"Digital Brands, LLC",,,6
6,Feb 3 2023 2:00PM,256632,12,Livs Products,,,1
7,Feb 3 2023 1:59PM,256631,19,ACG North America LLC,,,2
8,Feb 3 2023 1:59PM,256630,10,ISDIN Corporation,,,12
9,Feb 3 2023 1:44PM,256628,19,"Emersa Waterbox, LLC",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 6 2023 8:30AM,256640,10,Emerginnova,1,,
1,Feb 3 2023 2:49PM,256639,10,Eywa Pharma Inc.,1,,
2,Feb 3 2023 2:40PM,256638,10,Bio-PRF,9,,
3,Feb 3 2023 2:20PM,256635,10,ISDIN Corporation,20,,
4,Feb 3 2023 2:20PM,256634,10,ISDIN Corporation,10,,
5,Feb 3 2023 2:11PM,256633,10,"Digital Brands, LLC",6,,
6,Feb 3 2023 2:00PM,256632,12,Livs Products,1,,
7,Feb 3 2023 1:59PM,256631,19,ACG North America LLC,2,,
8,Feb 3 2023 1:59PM,256630,10,ISDIN Corporation,12,,
9,Feb 3 2023 1:44PM,256628,19,"Emersa Waterbox, LLC",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 6 2023 8:30AM,256640,10,Emerginnova,1,,
1,Feb 3 2023 2:49PM,256639,10,Eywa Pharma Inc.,1,,
2,Feb 3 2023 2:40PM,256638,10,Bio-PRF,9,,
3,Feb 3 2023 2:20PM,256635,10,ISDIN Corporation,20,,
4,Feb 3 2023 2:20PM,256634,10,ISDIN Corporation,10,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 6 2023 8:30AM,256640,10,Emerginnova,1.0,NaN,NaN
1,Feb 3 2023 2:49PM,256639,10,Eywa Pharma Inc.,1.0,NaN,NaN
2,Feb 3 2023 2:40PM,256638,10,Bio-PRF,9.0,NaN,NaN
3,Feb 3 2023 2:20PM,256635,10,ISDIN Corporation,20.0,NaN,NaN
4,Feb 3 2023 2:20PM,256634,10,ISDIN Corporation,10.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 6 2023 8:30AM,256640,10,Emerginnova,1.0,0.0,0.0
1,Feb 3 2023 2:49PM,256639,10,Eywa Pharma Inc.,1.0,0.0,0.0
2,Feb 3 2023 2:40PM,256638,10,Bio-PRF,9.0,0.0,0.0
3,Feb 3 2023 2:20PM,256635,10,ISDIN Corporation,20.0,0.0,0.0
4,Feb 3 2023 2:20PM,256634,10,ISDIN Corporation,10.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2822873,87.0,5.0,3.0
12,513255,1.0,1.0,0.0
15,256592,5.0,1.0,3.0
19,1026483,14.0,13.0,15.0
21,769800,3.0,0.0,0.0
22,513187,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2822873,87.0,5.0,3.0
1,12,513255,1.0,1.0,0.0
2,15,256592,5.0,1.0,3.0
3,19,1026483,14.0,13.0,15.0
4,21,769800,3.0,0.0,0.0
5,22,513187,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,87.0,5.0,3.0
1,12,1.0,1.0,0.0
2,15,5.0,1.0,3.0
3,19,14.0,13.0,15.0
4,21,3.0,0.0,0.0
5,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,87.0
1,12,Released,1.0
2,15,Released,5.0
3,19,Released,14.0
4,21,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,21,22
Status,,,,,,
Completed,3.0,0.0,3.0,15.0,0.0,0.0
Executing,5.0,1.0,1.0,13.0,0.0,0.0
Released,87.0,1.0,5.0,14.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,21,22
0,Completed,3.0,0.0,3.0,15.0,0.0,0.0
1,Executing,5.0,1.0,1.0,13.0,0.0,0.0
2,Released,87.0,1.0,5.0,14.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,21,22
0,Completed,3.0,0.0,3.0,15.0,0.0,0.0
1,Executing,5.0,1.0,1.0,13.0,0.0,0.0
2,Released,87.0,1.0,5.0,14.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()